<a href="https://colab.research.google.com/github/juliajung11/Basic-Text-Analysis-using-R/blob/master/Moving_from_words_to_phrases_part_1_of_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Moving from words to phrases when doing NLP
- [Abe Handler](https://www.abehandler.com/) University of Colorado, Boulder
- [Shufan Wang](https://people.cs.umass.edu/~shufanwang/) University of  Massachusetts, Amherst

## Introduction

If you have found this tutorial (companion slides [here](https://docs.google.com/presentation/d/1C5O0EdgM33SO1KlCk90rW1g7nuf2264XcuYPZB-b7bI/edit?usp=sharing)), you have probably done NLP projects where you (a) start with documents, (b) break them into individual words, and then (c) use computation to draw conclusions about the words in the documents. For instance, in your last project, maybe you took a collection of documents, broke the documents into individual words and then ran a topic model to find groups of words that tended to appear together in the documents. 

Breaking documents into individual words implicitly represents text in terms of single-word units called **unigrams**. The unordered collection of all the unigrams in a document is often called a   **bag of words** (see [Jurafsky and Martin](https://web.stanford.edu/~jurafsky/slp3/4.pdf)). Representing text using the unigram bag of words has many advantages. For one, analyzing unigrams is easy and fast; you can take a big text document and break it into a bunch of single-word observations, so you can observe useful statistical properties from the text. 

However, breaking documents into single words for downstream analysis does have downsides. One limitation is that some concepts or linguistic units within documents consist of multiple words, and so get lost or discarded when you using a unigram representation. For instance, the string "immigration hearing" refers to a particular legal proceeding. If you break this string into single word units "immigration" and "hearing", and put each of these unigrams to your bag of words, your representation of the text does not really represent the concept "immigration hearing".  That means when you draw conclusions about lexical units during downsteam analysis, you won't be able to draw conclusions about "immigration hearing" (e.g., what liberal and conservative judges say about an "immigration hearing"). 

For this reason, it sometimes may make sense for you to analyze groups of words instead of unigrams. This tutorial will show you how to do NLP with groups of words, which we will call **phrases** or **multi-word expressions**. We will show how to (A) extract phrases from documents and (B) use these phrases for downstream analysis. 

## High-level takeaways from this code

1. **You can use phrases when you do NLP.** There are many existing tools and methods for extracting phrases (e.g. [PyATE](https://github.com/kevinlu1248/pyate)). In this tutorial, we will explore using phrases extracted via Python package [phrasemachine](https://github.com/slanglab/phrasemachine) which you can install using `pip install phrasemachine`. Phrasemachine is based on the method described in the paper [Bag of What](https://aclanthology.org/W16-5615.pdf).
2. **You can be creative and define phrases in a way that makes sense for your problem.**  `phrasemachine` uses a particular regular expression over part-of-speech tags to extract phrases. The particular phrasemachine patterns are often useful. But in your work, think of other kinds of phrasal patterns you might want to extract using other regular expressions. For instance, at the end of the notebook I show how to use phrasemachine to extract phrases matching the regex `(theory)(of)D*(A|N)*N)` which extracts things like (e.g. "theory of federal supremacy"). (Note: this example is a regex over both words and part-of-speech tags. I use a hack-y way to extract phrases matching this regex at the end of the code, but there are other ways to do this too.)

In [ ]:
! wget https://zissou.infosci.cornell.edu/convokit/datasets/supreme-corpus/cases.jsonl -O cases.jsonl
! pip install phrasemachine
! pip install tqdm

--2022-02-17 16:06:49--  https://zissou.infosci.cornell.edu/convokit/datasets/supreme-corpus/cases.jsonl
Resolving zissou.infosci.cornell.edu (zissou.infosci.cornell.edu)... 128.253.51.178
Connecting to zissou.infosci.cornell.edu (zissou.infosci.cornell.edu)|128.253.51.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13337468 (13M) [application/octet-stream]
Saving to: ‘cases.jsonl’

cases.jsonl         100%[===================>]  12.72M  40.6MB/s    in 0.3s    

2022-02-17 16:06:49 (40.6 MB/s) - ‘cases.jsonl’ saved [13337468/13337468]

     |████████████████████████████████| 2.7 MB 3.2 MB/s 
  Created wheel for phrasemachine: filename=phrasemachine-1.0.7-py3-none-any.whl size=2694883 sha256=757f22ca024a81e25faeea75383d787a85aec924c8dd0184dadfe669c8003f3a
  Stored in directory: /root/.cache/pip/wheels/49/d9/e6/e8948b0664fc1e5135444099525ae3f67cebc0bcddf0a7b453
Successfully built phrasemachine


In [ ]:
! pip install convokit==2.5.2 # later versions seem to have some issue w/ a torch dependency. If you go to runtime-> restart runtime this seems to work (AH: Feb 15, 22)

     |████████████████████████████████| 167 kB 2.6 MB/s 
     |████████████████████████████████| 6.0 MB 4.1 MB/s 
     |████████████████████████████████| 1.5 MB 18.1 MB/s 
     |████████████████████████████████| 235 kB 18.3 MB/s 
     |████████████████████████████████| 53 kB 1.4 MB/s 
     |████████████████████████████████| 174 kB 17.0 MB/s 
     |████████████████████████████████| 748 kB 20.0 MB/s 
     |████████████████████████████████| 628 kB 19.5 MB/s 
     |████████████████████████████████| 451 kB 19.0 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 10.1 MB 17.9 MB/s 
     |████████████████████████████████| 181 kB 20.3 MB/s 
  Created wheel for convokit: filename=convokit-2.5.2-py3-none-any.whl size=203280 sha256=20cda9f05d4ffe394ba48ddd51ea95166df7d6bc3a2dee1a0bdc93714fa19e52
  Stored in directory: /root/.cache/pip/wheels/43/28/54/f09f7f6ffd8fe1d6aafeb89b36609f9b2334dbfe5179dd9eb9
  Created wheel for emoji: filename=emoji-1.6.3

In [ ]:
from convokit import Corpus, download # install the corpus here
corpus = Corpus(filename=download("supreme-corpus")) # download the corpus

### High-level plan

- Extract phrases from supreme court oral arguments
- Assign partisanship scores to phrases based on how frequently they occur in comments from liberal vs. conservative justices
- Plot them by frequency (x-axis) and partisanship (y-axis, color and size)
- Based on [Fightin' words](http://languagelog.ldc.upenn.edu/myl/Monroe.pdf) from Monroe, Colaresi and Quinn.

### High-level result

(Run code in cells below first or `make_plot` won't run)

In [ ]:
make_plot(tops_words)  # this line needs the code below to run. I wanted to start with the high-level takeaway. Run the cells below to get this line to work

alt.LayerChart(...)

In [ ]:
make_plot(tops_phrases) # this line needs the code below to run. I wanted to start with the high-level takeaway. Run the cells below to get this line to work

alt.LayerChart(...)

#### Unigram bag of words

Let's start with a single (short) document and break it into a unigram bag of words. It's easy to find packages for this online, but we will just use vanilla Python for this. A few notes:

- We will define words using a whitespace delimiter below, but note there are also other better ways to do [tokenization](https://web.stanford.edu/~jurafsky/slp3/ed3book.pdf).
- Note that a [bag](https://en.wikipedia.org/wiki/Multiset) is a set that allows duplicates; notice that the word `a` appears two times in the `unigram_bag_of_words`.
- Note that each item in our bag is a unigram (single word)

In [ ]:
from collections import defaultdict
document =  "Solyndra received a loan guarantee. The Department of Energy offered the guarantee.".replace(".","")
unigram_bag_of_words = defaultdict(int)
for word in document.split():
    unigram_bag_of_words[word] += 1

unigram_bag_of_words

defaultdict(int,
            {'Department': 1,
             'Energy': 1,
             'Solyndra': 1,
             'The': 1,
             'a': 1,
             'guarantee': 2,
             'loan': 1,
             'of': 1,
             'offered': 1,
             'received': 1,
             'the': 1})

### Discussion: what are some phrases that get missed?

### Adding phrases to the bag of words

In [ ]:
import phrasemachine
from tqdm.notebook import tqdm
text = "Solyndra received a loan guarantee. The Department of Energy offered the guarantee.".replace(".","")
out = phrasemachine.get_phrases(text)
out

{'counts': Counter({'department of energy': 1, 'loan guarantee': 1}),
 'num_tokens': 12}

In [ ]:
# here we are adding the phrases to the unigram bag of words

enriched_bag_of_words = unigram_bag_of_words
for phrase in out["counts"]:
    enriched_bag_of_words[phrase] = out['counts'][phrase]

enriched_bag_of_words

defaultdict(int,
            {'Department': 1,
             'Energy': 1,
             'Solyndra': 1,
             'The': 1,
             'a': 1,
             'department of energy': 1,
             'guarantee': 2,
             'loan': 1,
             'loan guarantee': 1,
             'of': 1,
             'offered': 1,
             'received': 1,
             'the': 1})

## Using phrases for downstream analysis

Now that we know how to extract phrases using phrasemachine, we will now see how to use such phrases for downstream analysis. Specifically, we will analyze the ideological orientation of words and phrases in U.S. Supreme Court Oral Arguments.

At a high level, we will ask: what kinds of things to liberal and conservative justices tend to bring up during oral arguments? What would you expect to liberals and convservatives to talk about?

##### Corpus
We will use the [`convokit`](https://convokit.cornell.edu/documentation/supreme.html) corpus of supreme court oral arguments. In this notebook, we will only examine comments from liberal and conservative justices from the years 2010-2019.

In this analysis, we will investigate which phrases are used by liberal (L) and conservative (C) justices. So we need a mapping of justices to ideologies, which we construct manually below.

In [ ]:
judge2ideology = {'j__john_g_roberts_jr': "C", 
                  'j__samuel_a_alito_jr': "C",
                  'j__ruth_bader_ginsburg': "L",
                  'j__sonia_sotomayor': "L",
                  'j__antonin_scalia': "L",
                  'j__stephen_g_breyer': "L",
                  'j__anthony_m_kennedy': "C",
                  'j__elena_kagan': "L",
                  'j__clarence_thomas': "C",
                  'j__neil_gorsuch': "C",
                  'j__brett_m_kavanaugh': "C"
                  }

We also build a set of all justices in the dataset.

In [ ]:
import json

def get_justices(input_file="cases.jsonl"):
    '''Get names of all justices in the dataset'''
    all_justices = set()
    with open(input_file, "r") as inf:
        for j in inf:
            j = json.loads(j)
            if j["votes"] is not None:
                for justice in j["votes"].keys():
                    all_justices.add(justice)
    return all_justices

all_justices = get_justices()

The next step is to extract words and phrases for the justices. In computing this information, we do two things to make the computational requirements more managable: 
1. We limit our analysis to court cases from 2010-2019, which is why you see `u.meta["case_id"][0:3] == "201"` below. 
2. We also only extract phrases from the first 500 characters of the utterance.

In [ ]:
utterances = [] # build a list of the utterances we are interested in
for u in tqdm(corpus.get_utterance_ids()):
    u = corpus.get_utterance(u)
    if u.speaker.id in all_justices and u.meta["case_id"][0:3] == "201":
        utterances.append(u)

  0%|          | 0/1700789 [00:00<?, ?it/s]

### Extracting phrases

The code below extracts and counts phrases from liberal and conservative justices

In [ ]:
from collections import defaultdict # https://docs.python.org/3/library/collections.html#collections.defaultdict
from tqdm.notebook import tqdm

justice2phrases = defaultdict(lambda: defaultdict(int))

for u in tqdm(utterances):
    phrases = phrasemachine.get_phrases(u.text[0:500])["counts"] # roughly 97.5% are less than 500 chars, and runs way faster
    for p in phrases:
        # we can filter out some filler/stop phrases here, e.g. when the record notes laughter
        if "justice" not in p and "mr." not in p and "minutes" not in p and "laugher" not in p:
            justice2phrases[judge2ideology[u.speaker.id]][p] += phrases[p]

phrasecounts = justice2phrases # this builds a dictionary of count of phrases by liberal/conservative judges

  0%|          | 0/78598 [00:00<?, ?it/s]

### Extracting words

The code below extracts and counts unigrams from liberal and conservative justices

In [ ]:
justice2words = defaultdict(lambda: defaultdict(int))

for u in tqdm(utterances):
    words = u.text[0:500].split() # this is a very simple tokenization strategy, see https://web.stanford.edu/~jurafsky/slp3/2.pdf for alternatives
    for word in words:
        # we can filter out some filler/stop phrases here, e.g. when the record notes laughter
        if "justice" not in word and "mr." not in word and "minutes" not in word and "laugher" not in word:
            justice2words[judge2ideology[u.speaker.id]][word] += 1

wordcounts = justice2words # this builds a dictionary of count of phrases by liberal/conservative judges

  0%|          | 0/78598 [00:00<?, ?it/s]

## Analyzing word use

Now that we have counted words and phrases from liberal and convservative justices, we will analyze differences in the political orientation of words and phrases. Specifically we will:
- Compute statistics about how frequently liberal and conservative justices use particular words and 
- Display this information on a plot for analysis 
- The partisanship score we will use in this notebook is $p(w|liberal) - p(w|conservative)$ which is assigned in `compute_phrase_scores`

Our approach is based on the [Fightin' Words](http://languagelog.ldc.upenn.edu/myl/Monroe.pdf) method from Monroe et al. Specifically, we will use the word importance score from Section 3.2.2 of Fightin' Words. If you are curious, the paper describes other word importance scores.

In [ ]:
import pandas as pd

def compute_normalize_counts(_countdict, n):
    '''
    normalize the count of words/phrases by total words (in the L or C wing)
    '''
    normalized_counts = defaultdict(lambda: defaultdict(int))

    for wing in _countdict.keys():
        for p in _countdict[wing]:
            normalized_counts[wing][p] = _countdict[wing][p]/n[wing]

    return normalized_counts

def compute_phrase_scores(normalized, _countdict):
    '''
    compute the count and partisanship scores of phrases
    '''
    df = []
    for wing in _countdict.keys():
        for phrase in _countdict[wing]:
            df.append({"score": normalized["L"][phrase] - normalized["C"][phrase], 
                    "phrase": phrase,
                    "count": _countdict["C"][phrase] + _countdict["L"][phrase]}) # http://languagelog.ldc.upenn.edu/myl/Monroe.pdf, 3.2.2
    return pd.DataFrame(df).drop_duplicates()

def getK(_df, k=20):
    '''
    Return the top k or bottom k items in a dataframe. This is just a convenience function
    '''
    if k > 0:
        return _df.sort_values("score")[0:k].copy()
    else:
        return _df.sort_values("score")[k:].copy()

def get_top_K_df(counts):
    '''Make a pandas dataframe of the top K partisan phrases for plotting'''
    countdict = counts

    n = {}  
    n["C"] = sum(countdict["C"].values())
    n["L"] = sum(countdict["L"].values())

    normalized_counts = compute_normalize_counts(countdict, n)

    df = compute_phrase_scores(normalized_counts, countdict)

    df = df[df["count"] < 200] # exclude high-count lexical items, roughly stop words

    # add an abolute value of the score
    df["score_abs"] = df["score"].apply(lambda x: abs(x))

    return pd.concat([getK(df, k=-20), getK(df, k=20)])


tops_phrases = get_top_K_df(phrasecounts)
tops_words = get_top_K_df(wordcounts)
tops_words["label"] = tops_words["phrase"].apply(lambda x: x.replace(".", ""))
tops_phrases["label"] = tops_phrases["phrase"]

In [ ]:
import altair as alt
import pandas as pd


def make_plot(source):

    height = 1000

    points = alt.Chart(source).mark_circle().encode(
        x='count:Q',
        y='score:Q',
        size='score_abs',
        color=alt.Color('score:Q', scale=alt.Scale(scheme='redyellowblue'))
    ).properties(
        width=1200,
        height=height
    )

    text = alt.Chart(source).mark_text(
        align='left',
        baseline='middle',
        dx=7
    ).encode(
        x='count:Q',
        y='score:Q',
        text='label'
    ).properties(
        width=1200,
        height=height
    )

    return points + text


In [ ]:
make_plot(tops_words) # this is the same line of code that runs at the top of the notebook. If you are running the cells from top-to-bottom this should run

alt.LayerChart(...)

### Discussion

- Comparing the unigram plot to the plot with phrases, what do you notice? 

- Which plot gives you a clearer sense what justices tend to talk about. This is sometimes called being more "[interpretable](https://arxiv.org/abs/1702.08608)".

### Defining phrases in your own way

It is also possible to define your own phrases for *your* task. Here is one example of how to do that based on phrasemachine. Note that this example mixes POS tags and words. The method shown below (based on phrasemachine) is somewhat hacky. 

There is also a more principled way of running a regex over POS tags and characters: (1) replace all words in your corpus with POS tags *except* the words you care about (2) run your regex over the replaced version of your corpus (3) extract phrases from the matches in the original corpus. That method is a little more involved and so is not shown here.

In [ ]:
import phrasemachine

utterances = [] # build a list of the utterances we are interested in
c = 0 

def theory_phrases(_text):
    '''
    Equivalent to this regex: (theory)(of)D*(A|N)*N)*
    
    To run this regex on your text, replace all words except "theory of" w. their POS tags
    '''
    for phrase in phrasemachine.get_phrases(_text)["counts"]:
        if phrase[0:len("theory of")] == "theory of":
            yield phrase

for u in tqdm(corpus.get_utterance_ids()):
    u = corpus.get_utterance(u)
    c += 1
    if c == 50000:
        break
    for phrase in theory_phrases(u.text):
        print(phrase)

  0%|          | 0/1700789 [00:00<?, ?it/s]

theory of the petitioner
theory of decisions
theory of the cases
theory of alter
theory of alter ego
theory of the matter
theory of milliken
theory of milliken versus
theory of milliken versus meyer
theory of the government
theory of burton-sutton
theory of congressional power
theory of the act
theory of the act as a disclosure
theory of the act as a disclosure statute
theory of the total
theory of the total for the four years
theory of indemnity
theory of the assignee
theory of the assignee of the cause
theory of the case
theory of lien
theory of lien priority
theory of the registrants
theory of the registrant
theory of the case
theory of this act
theory of the proceeding
theory of the petitioner
theory of deportation
theory of the decision
theory of the decision below
theory of the jurisdiction
theory of the jurisdiction in the western district
theory of the case
theory of the case whatsoever
theory of federal supremacy
theory of a fine
theory of a fine in a criminal case
theory of i